# Install pakcages

In [ ]:
!pip install pymysql    # mysql 연동을 위한 패키지
!pip install psycopg2   # postgresql 연동을 위한 패키지, 패키지 에러가 나면 psycopg2-binary로 설치
!pip install sqlalchemy # sqlalchemy 설치를 위한 패키지
!pip install pymongo    # mongodb 연동을 위한 패키지

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

import psycopg2
import pymysql
import pymongo
import pandas as pd

from sqlalchemy import create_engine

# DataBase

## MongoDB

비관계형 데이터베이스 관리 시스템(DBMS)으로 No-SQL의 한 종류 <br>
아래의 특징 존재 
1. 문서 지향 데이터베이스
데이터를 문서 형태로 저장 <br>
문서는 json 형식으로 저장
문서 기반 저장 방식은 데이터의 중첩 및 계층적 구조 지원
2. 스키마 유연성
schema-less 데이터베이스로, 데이터 모델이 유연하고 스키마를 사전에 정의할 필요 없음 <br>
각 문서는 다른 구조를 가질 수 있어 데이터 모델링이 더 유연 <br>
3. 확장성
수평적 확장을 지원하여, 데이터 샤딩(sharding)을 통해 대량의 데이터를 여러 서버에 분산하여 저장 가능 <br>

<br>

mongo compass
- MongoDB 데이터를 쿼리, 최적화 및 분석할 수 있는 무료 대화형 도구
- 중요한 인사이트을 찾아내고, 드래그 앤 드롭으로 파이프라인을 구축하는 등 여러 가지 기능을 제공. <br>
- link: https://www.mongodb.com/products/tools/compass


### 기본 커맨드

접속
```
bash     # 1
mongosh  # 2
```
<br>

종료
```
quit
```
<br>

DB 조회
```
show databases;
```
<br>

유저 조회
```
show users;
```  
<br>

테이블 조회
```
show collections;
```
<br>

테이블 접근
```
db.*collection name*
```

### collection 커맨드

생성: 별도의 생성 커맨드 없이 데이터 삽입 시 자동 생성
```
db.*collection name*.insertOne({ name: "Alice", age: 30 });
```
<br>

삭제
```
db.*collection name*.drop();
```
<br>

In [ ]:
# nosql에서는 document가 row이다.
# nosql에서는 collection이 스키마이다.

### CRUD

<span style="font-size: 18px;">Create</span>

```bash
# 단일 문서 삽입
db.*collection_name*.insertOne({
    name: "Alice",
    age: 30
});

# 여러 문서 삽입
db.*collection_name*.insertMany([
    { name: "Bob", age: 25 },
    { name: "Charlie", age: 35 }
]);
```

<br>

<span style="font-size: 18px;">Read</span>

```bash
# 특정 조건에 맞는 문서 조회
db.*collection_name*.find({ name: "Alice" });

# 첫 번째 일치 문서 조회
db.*collection_name*.findOne({ name: "Alice" });
```

<br>

<span style="font-size: 18px;">Update</span>

```bash
# 단일 문서 업데이트
db.*collection_name*.updateOne(
    { name: "Alice" },
    { $set: { age: 31 } }
);

# 여러 문서 업데이트
db.*collection_name*.updateMany(
    { age: { $lt: 30 } },
    { $set: { status: "young" } }
);
```

<br>

<span style="font-size: 18px;">Delete</span>

```bash
# 단일 문서 삭제
db.*collection_name*.deleteOne({ name: "Alice" });

# 여러 문서 삭제
db.*collection_name*.deleteMany({ age: { $lt: 30 } });
```

### 유저 커맨드

user 관리는 admin db에서 진행

```bash
use admin
```

<br>

<span style="font-size: 18px;">유저 생성</span>

```bash
db.createUser({
    user: "*user_name*",
    pwd: "*user_password*",
    roles: [
        { role: "readWrite", db: "*db_name*" },
        { role: "dbAdmin", db: "*db_name*" }
    ]
});
```

<br>

roles: 사용자의 역할 및 권한 <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;readWrite: DB 읽기 및 쓰기 권한 부여 <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;dbAdmin: DB 관리 작업을 수행 권한 <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;db: 권한을 부여할 데이터베이스 <br>

<br>

<span style="font-size: 18px;">유저 정보 변경</span>

```bash
db.updateUser("*user_name*", {
    pwd: "new_password",
    roles: [
        { role: "readWrite", db: "*db_name*" },
        { role: "dbAdmin", db: "*db_name*" }
    ]
});
```

<br>

<span style="font-size: 18px;">유저 삭제</span>

```bash
db.dropUser("*user_name*");
```

### 쿼리 연산자 및 필터

```bash
$eq: 일치하는 문서 찾기
$ne: 일치하지 않는 문서 찾기
$gt: 지정된 값보다 큰 값 찾기
$lt: 지정된 값보다 작은 값 찾기
$gte: 지정된 값보다 크거나 같은 값 찾기
$lte: 지정된 값보다 작거나 같은 값 찾기
$in: 배열 내의 값 중 하나와 일치하는 문서 찾기
$and: 여러 조건을 만족하는 문서 찾기
$or: 여러 조건 중 하나를 만족하는 문서 찾기
```

<br>

<span style="font-size: 18px;">Example</span>

```bash
// age가 25 이상 35 이하인 문서 찾기
db.*collection_name*.find({
    age: { $gte: 25, $lte: 35 }
});

// age가 25 미만 또는 35 초과인 문서 찾기
db.*collection_name*.find({
    $or: [
        { age: { $lt: 25 } },
        { age: { $gt: 35 } }
    ]
});
```

### 문자열 관련 쿼리

<span style='font-size: 18px'> Exact Matching </span>

```bash
db.*collection_name*.find({ name: "Alice" });
```

<br>

<span style='font-size: 18px'> Regex </span>

```bash
db.*collection_name*.find({ name: { $regex: "Alice" } });   # name에 Alice가 포함된 document
db.*collection_name*.find({ name: { $regex: "alice", $options: "i" } });  # name에 alice가 포함된 document인데 대소문자 구분 없음
db.*collection_name*.find({ name: { $regex: "^Alice" } });  # name이 Alice로 시작하는 document
db.*collection_name*.find({ name: { $regex: "Alice$" } });  # name이 Alice로 끝나는 document
db.*collection_name*.find({ $expr: { $gt: [{ $strLenCP: "$name" }, 5] } }); # name의 길이가 5보다 큰 document
```

### 집계함수

파이프라인 방식으로 집계함수가 동작하며, 각 파이프라인 단계는 문서를 변형하거나 필터링하는 작업을 수행 <br>
각 단계의 출력은 다음 단계의 입력으로 사용 <br>

```bash
$match: 조건에 맞는 문서를 필터링           # where
$group: 문서를 그룹화하고 집계 함수를 적용    # gropuby
$sort: 문서를 정렬                       # sort
$project: 문서의 필드를 포함하거나 제외      # select
$limit: 문서 수 제한                    # limit
$skip: 문서 스킵                        # offset
$lookup: 다른 컬렉션과 조인               # join
```

<br>

<span style='font-size: 18px'> Example </span>

```bash
db.*collection_name*.aggregate([
    { $match: { age: { $gte: 25 } } },
    { $group: { _id: "$department", totalSalary: { $sum: "$salary" } } },
    { $sort: { totalSalary: -1 } }
]);
```

#### match

문서의 특정 조건을 만족하는 문서만 선택

```bash
db.*collection_name*.aggregate([
    { $match: { age: { $gte: 25 } } }
]);
```

<br>

#### group

문서를 그룹화하고 집계 함수를 적용합니다

```bash
db.*collection_name*.aggregate([
    { $group: { _id: "$department", totalSalary: { $sum: "$salary" } } }
]);
```



#### sort

문서 정렬

```bash
db.*collection_name*.aggregate([
    { $sort: { age: -1 } }  # (1: 오름차순, -1: 내림차순)
]);
```

#### project

문서에서 특정 필드만 포함하거나 제외

```bash
db.*collection_name*.aggregate([
    { $project: { name: 1, age: 1, _id: 0 } }
]);
```

#### limit

문서 출력 수 제한

```bash
db.*collection_name*.aggregate([
    { $limit: 10 }
]);
```

#### skip

문서 수를 건너 뜀 (n개 document 이후 나머지 문서 반환)

```bash
db.*collection_name*.aggregate([
    { $skip: 5 }
]);
```

#### lookup

다른 컬렉션과 조인

```bash
db.orders.aggregate([
    { $lookup: {
        from: "products",
        localField: "productId",
        foreignField: "_id",
        as: "productDetails"
    } }
]);
```

<br>

orders 컬렉션의 productId 필드를 products 컬렉션의 _id 필드와 join 후 <br>
productDetails 필드에 결과를 포함

## Database Link in Python

### psycopg2

설치
```cmd
pip install psycopg2
pip install psycopg2-binary #위의 설치 에러 시 이 코드 실행
```
<br>

사용법
```python
conn = psycopg2.connect(
  host='address',
  dbname='database name',
  user='user name',
  password='password',
  port=port
)

cur = conn.cursor()
```

<br>

CRUD
```python
cur.execute(f'INSERT INTO test (id, press_name) VALUES ({id}, {press_name});')
conn.commit() # CREATE

cur.execute('SELECT * FROM test;')
result_one = cur.fetchone() # READ
result_many = cur.fetchmany() 
result_all = cur.fetchall() 

cur.execute(f'UPDATE test SET press_name={press_name} WHERE id > 5')
conn.commit() # UPDATE

cur.execute('DELETE FROM test WHERE press_name LIKE %조선%;')
conn.commit() # DELETE
```
<br>

pandas 내의 method를 통해서도 사용 가능
```python
pd.read_sql('SELECT * FROM test', conn)
```
<br>

Closer <br>
사용 후 연결 해제 <br>
```python
conn.close()
```
<br>

아래와 같이 사용 가능
```python
with conn.cursor() as cur:
  cur.execute(query)

conn.close()

with psycopg2.connect():
  with conn.cursor() as cur:
    cur.execute(query)
```

### pymysql

설치

```cmd
pip install pymysql
```
<br>

사용법
```python
conn = pymysql.connect(
  host='address',
  db='database name',
  user='user name',
  password='password',
  port=port,
  charset='utf8',
  autocommit=True,                          # optional, update, delete시 자동 commit
  cursorclass=pymysql.cursors.DictCursor,   # optional, db조회 시 결과를 dictionary로 반환
)

cur = conn.cursor()
```

<br>

CRUD
```python
cur.execute(f'INSERT INTO test (id, press_name) VALUES ({id}, {press_name});')
conn.commit() # CREATE

cur.execute('SELECT * FROM test;')
result_one = cur.fetchone() # READ
result_many = cur.fetchmany() 
result_all = cur.fetchall() 

cur.execute(f'UPDATE test SET press_name={press_name} WHERE id > 5')
conn.commit() # UPDATE

cur.execute('DELETE FROM test WHERE press_name LIKE %조선%;')
conn.commit() # DELETE
```
<br>

pandas 내의 method를 통해서도 사용 가능
```python
pd.read_sql('SELECT * FROM test', conn)
```
<br>

Closer <br>
사용 후 연결 해제 <br>
```python
conn.close()
```
<br>

아래와 같이 사용 가능
```python
with conn.cursor() as cur:
  cur.execute(query)

conn.close()

with pymysql.connect():
  with conn.cursor() as cur:
    cur.execute(query)
```

### sqlalchemy

설치
```cmd
pip install sqlalchemy
```
<br>

사용법

```python
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, Text, String, DateTime

# postgresql
db_url = f'postgres+psycopg2://{USERNAME}:{PASSWORD}@{DB_HOST}:{PORT}/{DB_NAME}'
# mysql

db_url = f'mysql+pymysql://{USERNAME}:{PASSWORD}@{DB_HOST}:{PORT}/{DB_NAME}'
engine = create_engine(db_url, echo=True)

test.to_sql(
  'schema',
  engine,
  if_exists='append', # replace: 덮어쓰기
  index=False,
  chunksize=5000,
  dtypes={              # optinal
    'id': Integer,
    'press_name': Text
  }
)
```

### pymongo

설치

```bash
pip install pymongo
```
<br>

사용법
```python
from pymongo import MongoClient

# 방법1. URI
uri = f'mongodb://{user_name}:{password}@{host}:{port}/{db_name}?authSource=admin'
client = MongoClient(uri)

# 방법2. parameter
client = MongoClient(
    host='localhost',
    port=27017,    
)

db = client['*database_name*']
collection = db['*collection_name*']
```

<br>

<span style='font-size: 18px'> DB 및 Collection 관련 커맨드 </span>

```python
client.list_database_names()                # database 조회
client.drop_database('*database name*')     # database 제거
db.list_collection_names()                  # collection 조회
db.drop_collection('*collection name*')     # collection 제거
```

<br>

<span style='font-size: 18px'> Create </span>

```python
# 단일 문서 삽입
result = *collection_name*.insert_one({'name': 'Alice', 'age': 25})

# 다수의 문서 삽입
result = *collection_name*.insert_many([{'name': 'Bob', 'age': 30}, {'name': 'Charlie', 'age': 35}])
```

<br>

<span style='font-size: 18px'> Read </span>

```python
# 단일 문서 조회
document = *collection_name*.find_one({'name': 'Alice'})

# 여러 문서 조회
documents = *collection_name*.find({'age': {'$gt': 25}})
```

<br>

<span style='font-size: 18px'> Update </span>

```python
# 단일 문서 업데이트
result = *collection_name*.update_one({'name': 'Alice'}, {'$set': {'age': 26}})

# 여러 문서 업데이트
result = *collection_name*.update_many({'age': {'$lt': 30}}, {'$set': {'status': 'young'}})
```

<br>

<span style='font-size: 18px'> Delete </span>

```python
# 단일 문서 삭제
result = *collection_name*.delete_one({'name': 'Alice'})

# 여러 문서 삭제
result = *collection_name*.delete_many({'age': {'$lt': 30}})
```

## Database Link to Streamlit

.streamlit 생성 -> secrets.toml 생성 후 연결하고자 하는 DB에 따라 secrets.toml에 내용 추가 <br>

### postgres

<span style="font-size: 18px;">secrets.toml</span>

```
[connections.postgresql]
dialect = "postgresql"
host = "localhost"
port = "5432"
database = "xxx"
username = "xxx"
password = "xxx"
```

<br>

<span style="font-size: 18px;">python script</span>


```python
import streamlit as st

conn = st.connection("postgresql", type="sql")
df = conn.query('SELECT * FROM *table_name*', ttl=600)
```

### mysql

<span style="font-size: 18px;">secrets.toml</span>

```
[connections.mysql]
dialect = "mysql"
host = "localhost"
port = 3306
database = "xxx"
username = "xxx"
password = "xxx"
query = { charset = "xxx" }
```

<br>

<span style="font-size: 18px;">python script</span>


```python
import streamlit as st

conn = st.connection("mysql", type="sql")
df = conn.query('SELECT * FROM *table_name*', ttl=600)
```

### mongo



<span style="font-size: 18px;">secrets.toml</span>

```
host = "localhost"
port = 27017
username = "xxx"
password = "xxx"
```

<br>

<span style="font-size: 18px;">python script</span>


```python
import streamlit as st

@st.cache_resource
def init_connection():
    return pymongo.MongoClient(**st.secrets["mongo"])

client = init_connection()

@st.cache_data(ttl=600)
def get_data():
    db = client.mydb
    items = db.mycollection.find()
    items = list(items)
    return items
```